항상 기뻐하라
쉬지 말고 기도하라
범사에 감사하라 이것이 그리스도 예수 안에서 너희를 향하신 하나님의 뜻이니라
성령을 소멸하지 말며
예언을 멸시하지 말고
범사에 헤아려 좋은 것을 취하고
악은 어떤 모양이라도 버리라 (살전5:16-22)

-------
<center><img src="https://github.com/idebtor/DSpy/blob/6b3676c3de29f85d2e7f30679676d1fd96d88b17/images/WelcomeToDataStructures.jpg?raw=true" width=1000></center>

__NOTE:__ The following materials have been compiled and adapted from the numerous sources including my own. Please help me to keep this tutorial up-to-date by reporting any issues or questions. Send any comments or criticisms to `idebtor@gmail.com` Your assistances and comments will be appreciated.

--------

## 단어 사다리 문제 (Word Ladder Problem)
: 단어 사다리 문제란 무엇인지, 그리고 이를 어떻게 해결하면 좋을지 배웁니다.

그래프 알고리즘을 배우기 위해 단어 사다리(Word Ladder)라는 퍼즐 문제를 다뤄보겠습니다. 먼저 __"FOOL"__ 이라는 단어를 __"SAGE"__ 로 바꿔보세요. 단어 사다리 퍼즐에서는 단어를 한 글자(알파벳)씩 점진적으로 바꿔야합니다. 이때, 한 글자로 바꾼 단어가 실제로 말이 되는 "단어"가 되어야 합니다. 이 게임은 "이상한 나라의 엘리스"의 저자인 Lewis Carroll이 1878년도에 제작한 게임 입니다. 위 FOOL 예제에 대한 많은 답 중 하나는 다음과 같을 수 있겠습니다.

``` bash
FOOL
POOL
POLL
POLE
PALE
SALE
SAGE
```

추가적으로 단어 사다리 퍼즐에는 많은 변칙들이 존재합니다. 예를 들어, 단어를 바꿀 수 있는 횟수의 제한을 준다거나, 중간에 특정 단어로 반드시 바꿔야 한다 등의 규칙을 추가하는 것입니다. 이번 강의에서 우리가 초점을 맞출 것은, 최초 단어에서 마지막 단어까지 필요한 최소한의 변환 횟수를 구하는 것입니다.

이 문제는 예상하셨듯이 그래프 알고리즘을 활용하여 풀 수 있습니다. 대략적인 개요는 다음과 같습니다.

* 단어들의 관계를 그래프로 표현한다.
* Breadth First Search(BFS, 너비 우선 탐색)으로 알려진 그래프 알고리즘을 활용하여, 시작 단어와 목표 단어 사이의 최적 path를 찾습니다.

### Step 1. 단어 사다리 그래프 만들기 (Word Ladder Graph) - Graph 클래스 만들기
우선 가장 먼저 해결해야할 문제는 방대한 양의 단어들을 하나의 그래프로 어떻게 나타낼 것이냐 입니다. 우리가 원하는 것은 많은 단어들 중, 철자가 하나만 다른 단어 사이에 edge를 형성하는 것입니다. 그렇게 된다면, 해당 그래프에서 어떠한 path라도 단어 사다리 문제의 해답이 될 수 있을 것입니다.

그림 1은 FOOL 에서 SAGE로 변환하는 작은 단어 사다리 그래프의 한 예입니다. 특별히 그래프가 undirected graph이며 edge에 weight가 부여되지 않았음을 기억하세요.

<center> <img src="https://github.com/idebtor/DSpy/blob/main/images/chap8/word_ladder_graph.png?raw=true" width="100%" title="Figure 1. A Small Word Ladder Graph"> <center><br>
<center>그림 1. Word Ladder Graph의 한 예<center><br>

단어 사다리 문제를 해결하기 위한 그래프를 만드는 것에는 다양한 접근 방법론들이 있습니다. 먼저 모두 같은 길이를 갖는 단어들의 리스트를 가지고 있다고 가정해봅시다. 그 다음, 리스트에 있는 모든 단어들을 비교해보아야 합니다. __우리의 최대 관심사는, 각 단어가 '서로 몇 글자씩 다르냐' 일 것입니다.__ 만약 두 단어가 한 글자(알파벳)밖에 차이나지 않는다면, 그 두 단어 사이에는 edge가 존재할 수 있게됩니다. 이러한 접근 방식은 단어의 수가 적을 경우 아주 잘 동작할 것이지만, 예를 들어 단어가 5,110개가 된다면, 단어를 비교하는 알고리즘이 $O(n^2)$를 갖기 때문에 $5,110^2$는 대략 2600만번 이상의 비교연산이 수행되게 됩니다.  

In [16]:
class Graph:
    def __init__(self, filename=None, delimiter=None):
        self._e = 0
        self._adj = dict()
        if filename is not None:
            with open(filename, 'r') as f:
                for line in f.read().splitlines():  # read all lines
                    names = line.split(delimiter)
                    if len(names)>1:                # if edge information given (input ex: VertexA VertexB)
                        for i in range(1, len(names)):
                            self.addEdge(names[0], names[1])
                    else:                           # if only vertex information given (input ex: VertexA)
                        self._adj[names[0]] = set()
                    
    def addEdge(self, v, w):
        if not self.hasVertex(v): self._adj[v] = set()
        if not self.hasVertex(w): self._adj[w] = set()
        if not self.hasEdge(v, w):
            self._e += 1
            self._adj[v].add(w)
            self._adj[w].add(v)
            
    def hasVertex(self, v):       return v in self._adj
    def hasEdge(self, v, w):      return w in self._adj[v]


### Step 1-2. Graph 클래스 보완하기

위에서 구현한 Graph 클래스는 주어진 파일을 읽고 각 단어를 vertex로 나타내는 최소한의 기능을 하도록 만들었습니다. 하지만, 그래프를 탐색하거나 그래프의 정보를 쉽고 효과적으로 파악하기 위해선 추가적인 함수들을 구현해주면 도움이 됩니다.

* `def __str__(self)`: Graph 객체의 출력을 통해 그래프의 연결 정보를 확인할 수 있게 한다.
* `def adjacentTo(self, v)`: 해당 Vertex의 인접 Vertex들을 찾아낸다.
* `def vertices(self)`: 해당 그래프에 존재하는 Vertex들을 반환한다.
* `def countV(self)`: 해당 그래프에 존재하는 Vertex들의 수를 반환한다.
* `def countE(self)`: 해당 그래프에 존재하는 Edge의 수를 반환한다.
* `def degree(self, v)`: 해당 Vertex의 degree를 반환한다.

In [17]:
class Graph:
    def __init__(self, filename=None, delimiter=None):
        self._e = 0
        self._adj = dict()
        if filename is not None:
            with open(filename, 'r') as f:
                for line in f.read().splitlines():  # read all lines
                    names = line.split(delimiter)
                    if len(names)>1:                # if two vertex names given(ex: VertexA VertexB)
                        for i in range(1, len(names)):
                            self.addEdge(names[0], names[1])
                    else:                           # if only one vertex name given(ex: VertexA)
                        self._adj[names[0]] = set()
                      
    def __str__(self):                                  # string representation of graph
        s = ''
        for v in self.vertices():
            s += v + ' '
            for w in self.adjacentTo(v):
                s += w + ' '
            s += '\n'
        return s
    
    def addEdge(self, v, w):                             # add edges v-w and w-v to graph
        if not self.hasVertex(v): self._adj[v] = set()
        if not self.hasVertex(w): self._adj[w] = set()
        if not self.hasEdge(v, w):
            self._e += 1
            self._adj[v].add(w)
            self._adj[w].add(v)
            
    def adjacentTo(self, v):  return iter(self._adj[v])  # iterable for neighbors of v
    def vertices(self):       return iter(self._adj)     # iterable for the vertices of graph
    def hasVertex(self, v):   return v in self._adj      # is v a vertex in graph
    def hasEdge(self, v, w):  return w in self._adj[v]   # is v-w and edge in graph
    def countV(self):         return len(self._adj)      # the number of vertices in graph
    def countE(self):         return self._e             # the number of edges in graph 
    def degree(self, v):      return len(self._adj[v])   # the number of neighbors of v

### Step 2. 바구니 만들기 -  `buildBuckets` 함수 만들기

다음 접근법은 위 성능을 개선시킬 수 있습니다. 우선, 네글자 단어 중 하나를 비워둔 큰 바구니들이 있다고 가정해봅시다. 예를 들어, 그림 2와 같이 "POP_"라는 바구니가 있을 수 있습니다. 이제 우리가 각 단어를 바구니에 넣을 때, "POPE"와 "POPS" 등의 단어가 모두 "POP_"로 매칭이 될 것입니다. 이렇게 모든 단어가 각 바구니로 나뉘어 들어가게 되면 아주 멋진 특징을 한가지 알 수 있게됩니다. 이는, 바구니 내의 모든 단어는 서로에게 모두 연결될 수 있다는 것입니다.

<center> <img src="https://github.com/idebtor/DSpy/blob/main/images/chap8/word_buckets.png?raw=true" width="60%" title="Figure 2. Word Buckets for Words That are Different by One letter"> <center><br>
<center>그림 2. 한 철자만 다른 단어 바구니들의 예<center><br>

파이썬에서는 위와 같은 아이디어를 딕셔너리로 구현할 수 있습니다. 각 바구니의 이름이 딕셔너리의 키(key)가 될 것이며, 딕셔너리의 값(value)은 단어가 될 것입니다. 이렇게 딕셔너리가 구성되게 되면 그래프를 쉽게 만들 수 있습니다. 딕셔너리의 각 단어를 vertex로 만들 때, 같은 키 값을 갖는 단어끼리는 edge를 부여하면 되는 것입니다.

먼저 위와 같은 바구니를 만들고, 각 바구니에 적합한 단어들을 넣는 `buildBuckets`함수를 만들어보세요. `buildBuckets`함수는 바구니들의 정보를 딕셔너리 형태로 담아 반환합니다.

In [24]:
class Graph:
    def __init__(self, filename=None, delimiter=None):
        self._e = 0
        self._adj = dict()
        if filename is not None:
            with open(filename, 'r') as f:
                for line in f.read().splitlines():  # read all lines
                    names = line.split(delimiter)
                    if len(names)>1:                # if two vertex names given(ex: VertexA VertexB)
                        for i in range(1, len(names)):
                            self.addEdge(names[0], names[1])
                    else:                           # if only one vertex name given(ex: VertexA)
                        self._adj[names[0]] = set()
    
    def addEdge(self, v, w):
        if not self.hasVertex(v): self._adj[v] = set()
        if not self.hasVertex(w): self._adj[w] = set()
        if not self.hasEdge(v, w):
            self._e += 1
            self._adj[v].add(w)
            self._adj[w].add(v)
            
            
    def __str__(self):
        s = ''
        for v in self.vertices():
            s += v + ' '
            for w in self.adjacentTo(v):
                s += w + ' '
            s += '\n'
        return s
    
    def hasVertex(self, v):       return v in self._adj
    def hasEdge(self, v, w):      return w in self._adj[v]
    def adjacentTo(self, v):      return iter(self._adj[v])
    def vertices(self):           return iter(self._adj)
    def countV(self):             return len(self._adj)
    def countE(self):             return self._e
    def degree(self, v):          return len(self._adj[v])
                        
    def buildBuckets(self):
        buckets = dict()
        for vertex in iter(self._adj):
            for i in range(len(vertex)):
                bucket = vertex[:i] + '_' + vertex[i+1:]
                if bucket in buckets:
                    buckets[bucket].append(vertex)
                else:
                    buckets[bucket] = [vertex]
        return buckets
  

### Step 3. 바구니를 활용한 Graph 그리기 - buildGraph 함수 만들기

이제 만들어진 `buildBuckets`라는 함수를 사용하여 얻은 단어들의 관계를 가지고 Edge를 이어 Graph를 그려보세요.

In [4]:
class Graph:
    def __init__(self, filename=None, delimiter=None):
        self._e = 0
        self._adj = dict()
        if filename is not None:
            with open(filename, 'r') as f:
                for line in f.read().splitlines():  # read all lines
                    names = line.split(delimiter)
                    if len(names)>1:                # if two vertex names given(ex: VertexA VertexB)
                        for i in range(1, len(names)):
                            self.addEdge(names[0], names[1])
                    else:                           # if only one vertex name given(ex: VertexA)
                        self._adj[names[0]] = set()
    
    def addEdge(self, v, w):
        if not self.hasVertex(v): self._adj[v] = set()
        if not self.hasVertex(w): self._adj[w] = set()
        if not self.hasEdge(v, w):
            self._e += 1
            self._adj[v].add(w)
            self._adj[w].add(v)
            
            
    def __str__(self):
        s = ''
        for v in self.vertices():
            s += v + ' '
            for w in self.adjacentTo(v):
                s += w + ' '
            s += '\n'
        return s
    
    def hasVertex(self, v):       return v in self._adj
    def hasEdge(self, v, w):      return w in self._adj[v]
    def adjacentTo(self, v):      return iter(self._adj[v])
    def vertices(self):           return iter(self._adj)
    def countV(self):             return len(self._adj)
    def countE(self):             return self._e
    def degree(self, v):          return len(self._adj[v])
                        
    def buildBuckets(self):
        buckets = dict()
        for vertex in iter(self._adj):
            for i in range(len(vertex)):
                bucket = vertex[:i] + '_' + vertex[i+1:]
                if bucket in buckets:
                    buckets[bucket].append(vertex)
                else:
                    buckets[bucket] = [vertex]
        return buckets
    
    def buildGraph(self):
        buckets = self.buildBuckets()
        for bucket in buckets.keys():
            for v1 in buckets[bucket]:
                for v2 in buckets[bucket]:
                    if v1 != v2:
                        self.addEdge(v1,v2)


### Step 4. 너비 우선 탐색 - BFS 클래스 만들기

이제 그래프가 구현되었으니 단어 사다리 문제를 최적의 경로로 풀어낼 수 있는 알고리즘을 생각해보도록 하겠습니다. 이번 강의에서는 가장 간단한 그래프 탐색 알고리즘 중 하나인, __Breadth First Search (BFS, 너비 우선 탐색)__ (이하 BFS) 알고리즘을 사용할 것입니다. 이 알고리즘은 이후 배울 다른 그래프 탐색 알고리즘의 프로토타입(prototype)으로도 사용될 것입니다.

그래프 $G$와 vertex $s$가 주어졌을 때, BFS는 $G$에서 $s$와 연결된 path에 포함되는 모든 vertex를 탐색합니다. 이때 BFS의 한가지 특징은, $s$로부터 $k+1$ 만큼 떨어진 vertex를 탐색하기 전에 $k$만큼 떨어진 모든 vertex를 탐색한다는 것입니다. 이는 쉽게 트리를 한 층(level)씩 그려나가는 것을 생각하면 감이 올 것입니다. BFS는 시작하는 vertex부터 손자(grandchild)노드를 생성하기 이전에 모든 자식(child)을 먼저 추가합니다.

1. 처음 발견한 vertex `s`를 Queue에 넣습니다.
2. Queue에서 vertex 하나를 dequeue하고, 해당 vertex와 인접한 vertex를 순차적으로 탐색합니다.
3. 처음 발견된 인접한 vertex는 Queue에 넣습니다. 이는 이후에 해당 노드를 탐색할 수 있도록 스캐줄링 하지만, `currentVert`의 인접 vertex들을 모두 탐색한 이후에 실행됩니다.
4. `s`의 predecessor는 현재 노드임을 나타내는 `currentVert`로 설정합니다.
5. `s`로의 거리를 `이전 노드의 거리 + 1`로의 거리로 설정합니다.
6. 2-5번의 과정을 반복합니다.

In [12]:
from dspy import Queue

class BFS:
    def __init__(self, graph, s):
        self._distTo = dict()
        self._pred = dict()
        queue = Queue()
        queue.enqueue(s)
        self._distTo[s] = 0
        self._pred[s] = None
        while not queue.is_empty():
            print(queue._items)
            v = queue.dequeue()
            for w in graph.adjacentTo(v):
                if w not in self._distTo:
                    queue.enqueue(w)
                    self._distTo[w] = 1 + self._distTo[v]
                    self._pred[w] = v
    
    def distanceTo(self, v):
        return self._distTo[v]
    
    def hasPathTo(self, v):
        return v in self._distTo
    
    def pathTo(self, v):
        path = []
        while v is not None:
            path += [v]
            v = self._pred[v]
        return reversed(path)


`BFS` 함수가 그래프에 따라 너비 우선 트리(breadth first tree)를 그림 1과 같이 생성하는 것을 보십시오. 

`fool`을 시작으로 그에 인접한 모든 노드를 트리에 추가합니다(`pool, cool, foil, foul`). 이 노드들은 각각 큐에 더해져서 나중에 탐색이 진행되도록 합니다. 그림 3은 해당 과정의 트리 모습과 큐의 모습을 그림으로 나타낸 것입니다.

<center> <img src="https://github.com/idebtor/DSpy/blob/main/images/chap8/bfs_first_step.png?raw=true" width="60%" title="Figure 3. The First Step in the Breadth First Search"> <center><br>
<center>그림 3. BFS의 첫번째 과정<center><br>

다음으로 `BFS`에서는 큐에서부터 다음 노드(pool)를 제거하여 같은 작업을 반복합니다. 그림 4는 다음 Edge 추가 과정을 그림으로 나타낸 것입니다.

<center> <img src="https://github.com/idebtor/DSpy/blob/main/images/chap8/bfs_second_step.png?raw=true" width="60%" title="Figure 4. The Second Step in the Breadth First Search"> <center><br>
<center>그림 4. BFS의 두번째 과정<center><br>

큐의 다음 vertex는 `cool`입니다. 이때 새로 추가될 노드는 없습니다. 이후 `BFS`알고리즘이 진행되며 탐색할 두 노드(`foil, foul`) 중 `foil`에는 `fail`을 추가하고 다른 하나에는 추가할 노드가 없습니다. 그림 5는 2번째 층의 모든 노드에 대해 탐색을 완료하고, 다음 층으로 확장한 모습을 그림으로 나타냅니다.

<center> <img src="https://github.com/idebtor/DSpy/blob/main/images/chap8/bfs_one_level.png?raw=true" width="60%" title="Figure 5. Breadth First Search Tree After Completing One Level"> <center><br>
<center>그림 5. 한 층을 거친 BFS 트리<center><br>

<center> <img src="https://github.com/idebtor/DSpy/blob/main/images/chap8/bfs_final_tree.png?raw=true" width="60%" title="Figure 6. Final Breadth First Search Tree"> <center><br>
<center>그림 6. 최종 BFS 트리<center><br>

BFS는 처음 볼때는 다소 복잡하게 느껴질 수 있습니다. 따라서 과정을 직접 그려보면서 따라가는 것이 좋을 것입니다. 그림 6은 그림 3의 BFS 트리를 최종적으로 확장한 모습을 나타낸 것입니다. 

BFS의 한가지 놀라운 점은, `fool - sage` 의 문제를 해결하면서 동시에 많은 하위 문제들의 해답을 구했다는 것입니다. 아무 도착 단어를 설정하더라도, BFS 트리의 링크를 올라가며 추적하면 가장 빠른 단어 사다리 path를 찾을 수 있게 됩니다. 코드 3은 한 단어에서 층을 올라가며 순차적으로 단어를 출력하는 함수를 구현한 것입니다.

In [9]:
if __name__ == "__main__": 
    graph = Graph("wordFile.txt")
    graph.buildGraph()

    print('no. of vertices:', graph.countV())
    print('no. of edges:', graph.countE())
    print('vertices:\n', [ v for v in graph.vertices() ])
    
    print('graph:\n', graph, sep='')
    
    print('graph._adj:\n', graph._adj)

no. of vertices: 15
no. of edges: 19
vertices:
 ['fool', 'pool', 'foil', 'foul', 'cool', 'poll', 'fail', 'fall', 'pole', 'pall', 'pope', 'pale', 'page', 'sale', 'sage']
graph:
fool foul foil cool pool 
pool fool cool poll 
foil fail fool foul 
foul fool foil 
cool fool pool 
poll pole pall pool 
fail foil fall 
fall pall fail 
pole pope poll pale 
pall pale poll fall 
pope pole 
pale pole page sale pall 
page sage pale 
sale sage pale 
sage page sale 

graph._adj:
 {'fool': {'foul', 'foil', 'cool', 'pool'}, 'pool': {'fool', 'cool', 'poll'}, 'foil': {'fail', 'fool', 'foul'}, 'foul': {'fool', 'foil'}, 'cool': {'fool', 'pool'}, 'poll': {'pole', 'pall', 'pool'}, 'fail': {'foil', 'fall'}, 'fall': {'pall', 'fail'}, 'pole': {'pope', 'poll', 'pale'}, 'pall': {'pale', 'poll', 'fall'}, 'pope': {'pole'}, 'pale': {'pole', 'page', 'sale', 'pall'}, 'page': {'sage', 'pale'}, 'sale': {'sage', 'pale'}, 'sage': {'page', 'sale'}}


In [15]:
import sys

graph = Graph("wordFile.txt")
graph.buildGraph()
print(graph)

source = 'fool'
paths = BFS(graph, source)

while True:
    print('SOURCE:', source)
    print('TARGET:', [ v for v in graph.vertices() ])
    line = input('Enter a target(q to quit): ')
    target = line.rstrip()
    if target == 'q': break
    if target not in graph.vertices(): continue
    if paths.hasPathTo(target):
        distance = paths.distanceTo(target)
        for v in paths.pathTo(target):
            print(' ' + v)
        print('distance:', str(distance))

fool foul foil cool pool 
pool fool cool poll 
foil fail fool foul 
foul fool foil 
cool fool pool 
poll pole pall pool 
fail foil fall 
fall pall fail 
pole pope poll pale 
pall pale poll fall 
pope pole 
pale pole page sale pall 
page sage pale 
sale sage pale 
sage page sale 

['fool']
['pool', 'cool', 'foil', 'foul']
['pool', 'cool', 'foil']
['fail', 'pool', 'cool']
['fail', 'pool']
['poll', 'fail']
['fall', 'poll']
['pall', 'pole', 'fall']
['pall', 'pole']
['pale', 'pope', 'pall']
['pale', 'pope']
['pale']
['sale', 'page']
['sage', 'sale']
['sage']
SOURCE: fool
TARGET: ['fool', 'pool', 'foil', 'foul', 'cool', 'poll', 'fail', 'fall', 'pole', 'pall', 'pope', 'pale', 'page', 'sale', 'sage']


Enter a target(q to quit):  sale


 fool
 pool
 poll
 pole
 pale
 sale
distance: 5
SOURCE: fool
TARGET: ['fool', 'pool', 'foil', 'foul', 'cool', 'poll', 'fail', 'fall', 'pole', 'pall', 'pope', 'pale', 'page', 'sale', 'sage']


Enter a target(q to quit):  q


-------
항상 기뻐하라
쉬지 말고 기도하라
범사에 감사하라 이것이 그리스도 예수 안에서 너희를 향하신 하나님의 뜻이니라
성령을 소멸하지 말며
예언을 멸시하지 말고
범사에 헤아려 좋은 것을 취하고
악은 어떤 모양이라도 버리라 (살전5:16-22)